In [8]:
## loading dependent packages
import pandas as pd
from pandas import DataFrame, Series
import glob
import os
import re

In [9]:
## defining the data directory
data_dir: str = '../data/train/'

In [10]:
## initializing the list to store the concatenatd data frames
merged_df_list: list[pd.DataFrame] = []

In [11]:
## locating all files that match the pattern input_2023_w[01-18].csv
input_files: list[str] = sorted(glob.glob(os.path.join(data_dir, 'input_2023_w*.csv')))

In [ ]:
## locating all files that match the pattern output_2023_w[01-18].csv
output_files = sorted(glob.glob(os.path.join(data_dir, 'output_2023_w*.csv')))

../data/train\output_2023_w01.csv


In [13]:
## defining the keys to join the corresponding input files on the output files
join_keys: list[str] = ['game_id', 'play_id', 'nfl_id']

In [18]:
## checking if the lists have the same length before zipping
if (len(input_files) != len(output_files)):
    print('Warning: The number of input files does not match the number of output files')

In [17]:
# Test W01 files
w01_input = pd.read_csv('../data/train/input_2023_w01.csv')
print(w01_input.columns)
w01_output = pd.read_csv('../data/train/output_2023_w01.csv')
print(w01_output.columns)

Index(['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y'],
      dtype='object')
Index(['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y'], dtype='object')


In [29]:
## using zip to iterate over the corresponding input and output files simultaneously
for input_file_path, output_file_path in zip(input_files, output_files):
    
    ## extracting the week identifier for clear matching
    match = re.search(r'(w\d{2})', os.path.basename(input_file_path))
    week = match.group(0) if match else 'Unknown_Week'
    # print(week)

    try:
        ## loading the dataframes
        print(f'Processing {week}: Loading {os.path.basename(input_file_path)} and {os.path.basename(output_file_path)}')
        input_df: pd.DataFrame = pd.read_csv(input_file_path, engine = 'python')
        output_df: pd.DataFrame = pd.read_csv(output_file_path, engine = 'python')
        # print(input_df.columns)
        # print(output_df.columns)

        ## renaming the overlapping target columns in the output file dataframe
        output_df = output_df.rename(columns = {'x': 'target_x',
                                                'y': 'target_y',
                                                'frame_id': 'target_frame_id'})
        # print(output_df.columns)

        ## merging the input and output dataframes on the specified keys through an inner merge to keep rows that exist in both
        ## the input and output files 
        merged_df: pd.DataFrame = pd.merge(
            input_df,
            output_df,
            on = join_keys,
            how = 'inner',
            suffixes = ('_input', '_output')
        )
        # print(f'Printing week {week} merged df')
        # print(merged_df.columns)

        ## adding the merged dataframe to the dataframe list
        merged_df_list.append(merged_df)

        ## verifying that all columns are present in the merged dataframe
        required_cols: list[str] = list(input_df.columns) + list(output_df.columns)
        missing_cols: list[str] = [col for col in required_cols if col not in merged_df.columns]

        if missing_cols:
            print(f'Warning: merged dataframe for {week} is missing expected columns: {missing_cols}')
            print(f'Successfully merged {week}, resulting in {len(merged_df)} rows')

    except FileNotFoundError:
        print(F'Error: File not found for {week}. Skipping.')
    except pd.errors.EmptyDataError:
        print(f'Error: One of the files for {week} is empty. Skipping.')
    except Exception as e:
        print(f'An unexpected error occurred during processing {week}: {e}')

print('\n--- Final Summary ---')
print(f'Processing complete. Total {len(merged_df_list)} merged dataframes stored in "merged_df_list."')


Processing w01: Loading input_2023_w01.csv and output_2023_w01.csv
Processing w02: Loading input_2023_w02.csv and output_2023_w02.csv
Processing w03: Loading input_2023_w03.csv and output_2023_w03.csv
Processing w04: Loading input_2023_w04.csv and output_2023_w04.csv
Processing w05: Loading input_2023_w05.csv and output_2023_w05.csv
Processing w06: Loading input_2023_w06.csv and output_2023_w06.csv
Processing w07: Loading input_2023_w07.csv and output_2023_w07.csv
Processing w08: Loading input_2023_w08.csv and output_2023_w08.csv
Processing w09: Loading input_2023_w09.csv and output_2023_w09.csv
Processing w10: Loading input_2023_w10.csv and output_2023_w10.csv
Processing w11: Loading input_2023_w11.csv and output_2023_w11.csv
Processing w12: Loading input_2023_w12.csv and output_2023_w12.csv
Processing w13: Loading input_2023_w13.csv and output_2023_w13.csv
Processing w14: Loading input_2023_w14.csv and output_2023_w14.csv
Processing w15: Loading input_2023_w15.csv and output_2023_w15